In [ ]:
import numpy as np
#import pandas as pd
import scipy.stats
import scipy.spatial
from sklearn.cross_validation import KFold
import random
from sklearn.metrics import mean_squared_error
from math import sqrt
import math
import warnings
import sys

In [ ]:
warnings.simplefilter("error")
users = 6036
items= 3952

In [ ]:
def readingfile(filename):
    f = open(filename,"r")
    data = []
    for row in f:
        r=row.split(",")
        e =[int(r[0]),int(r[1]),int(r[2])]
        data.append(e)
    return data
recommend_data = readingfile("ratings_new.csv")

In [ ]:
def similarity_user(data):
    user_cos_sim = np.zeros((users,users))
    user_jac_sim = np.zeros((users,users))
    user_pea_sim = np.zeros((users,users))
    for user1 in range(users):
        for user2 in range(users):
            if np.count_nonzero(data[user1]) and np.count_nonzero(data[user2]):
                user_cos_sim[user1][user2] = 1- scipy.spatial.distance.cosine(data[user1],data[user2])
                user_jac_sim[user1][user2] = 1- scipy.spatial.distance.jaccard(data[user1],data[user2])
            try:
                if not math.isnan(scipy.stats.pearsonr(data[user1],data[user2])[0]):
                    user_pea_sim[user1][user2] = scipy.stats.pearsonr(data[user1],data[user2])[0]
                else:
                    user_pea_sim[user1][user2] = 0
            except:
                    user_pea_sim[user1][user2] = 0
            #print(str(user1) + "," + str(user2) + "," + str(user_cos_sim[user1][user2]) + "," + str(user_cos_sim[user1][user2]) + "," + str(user_pea_sim[user1][user2]) + "\n")       
    return user_cos_sim,user_jac_sim,user_pea_sim

In [ ]:
def cross_validation(data):
    k_fold = KFold(n=len(data),n_folds =10)
    #print(k_fold)
    mat=np.zeros((users,items))
    for e in data[:1]:
        mat[e[0]-1][e[1]-1]=e[2]
    user_cos_sim,user_jac_sim,user_pea_sim = similarity_user(mat)
    return(mat)

cross_data=cross_validation(recommend_data)
cross_data

In [ ]:
def predictRating(recommend_data):
    M, sim_user = crossValidation(recommend_data)
    f = open(sys.argv[2],"r")
    toBeRated = {"user":[], "item":[]}
    for row in f:
        r = row.split(',')
        toBeRated["item"].append(int(r[1]))
        toBeRated["user"].append(int(r[0]))
    f.close()
    pred_rate = []
    fw_w = open('result1.csv','w')
    l = len(toBeRated["user"])
    for e in range(l):
        user = toBeRated["user"][e]
        item = toBeRated["item"][e]
        pred = 3.0
        #user-based
        if np.count_nonzero(M[user-1]):
            sim = sim_user[user-1]
            ind = (M[:,item-1] > 0)
            #ind[user-1] = False
            normal = np.sum(np.absolute(sim[ind]))
            if normal > 0:
                pred = np.dot(sim,M[:,item-1])/normal

        if pred < 0:
            pred = 0recommend_data = readingfile("ratings_new.csv")
predictRating(recommend_data)
        if pred > 5:
            pred = 5
        pred_rate.append(pred)
        print str(user) + "," + str(item) + "," + str(pred)

        fw_w.write(str(pred) + "\n")recommend_data = readingfile("ratings_new.csv")
predictRating(recommend_data)
        fw_w.close()

In [ ]:
predictRating(recommend_data)